<a href="https://colab.research.google.com/github/EvgeniyStrizhak/My-master-s-thesis/blob/main/Data_pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data pre-processing

This notebook downloads datasets from the [Bundesbank database](https://www.bundesbank.de/), merges and processes them, and uploads the resulting features to a GitHub repository as `features.csv`. It also includes oil price data retrieved from Yahoo Finance.

Descriptions and sources of all features can be found [here](https://github.com/EvgeniyStrizhak/My-master-s-thesis/blob/a5d137148423b66a3dbeb3045b7e776677ed5838/row_datasets/files_info.json).

The target variable is also downloaded from Yahoo Finance. The notebook fetches this data, processes it, and uploads it to GitHub as `target.csv`.

This pipeline allows for flexible configuration—it can be easily added or modified indicators from the Bundesbank by editing the `files_info.json` metadata file.

Note: Some indicators, like GDP, are reported quarterly, while the model forecasts monthly. In such cases, the quarterly values are repeated across all months of the quarter.

## Overview of the Data Pipeline

### Feature Processing

- Select the [data period](#scrollTo=JjFq39LVCj05&line=1&uniqifier=1)
- [Download WTI oil prices from Yahoo Finance](#scrollTo=K5rYrkuzys9v&line=1&uniqifier=1).  
  (Note: WTI was used instead of Brent due to a longer available time range)
- Apply the following preprocessing [steps](#scrollTo=JxhxrrleWbTn&line=1&uniqifier=1):
    1. Convert data types and select year/month
    2. Select the target column
    3. Apply filtering
    4. Aggregate daily data to monthly by selecting the last price of each month

- [Download data from the Bundesbank](#scrollTo=eGUR6eVrzsvG&line=1&uniqifier=1)
- Each feature from the Bundesbank has a separate dataset and URL. All URLs and metadata are stored in [this JSON file](https://github.com/EvgeniyStrizhak/My-master-s-thesis/blob/a5d137148423b66a3dbeb3045b7e776677ed5838/row_datasets/files_info.json)
- For each dataset, the following preprocessing [steps](#scrollTo=7w6WQ3F8jQJP&line=3&uniqifier=1) are applied:
    1. `filter_rows`
    2. `split_date_column`
    3. `drop_unnecessary_columns`
    4. `rename_columns`
    5. `convert_dates_to_int`
    6. `filter_by_year`
    7. `convert_feature_to_float`, `file_name`
    8. `reset_index`

- [Merge](#scrollTo=DHDA1Gm41HD2&line=1&uniqifier=1) Bundesbank data with Yahoo Finance features
- [Upload](#scrollTo=k3Frz06-7t6P&line=2&uniqifier=1) final dataset to the repository using the [upload_file function](#scrollTo=AkRCJYSmzsPL&line=14&uniqifier=1)

### Target Processing

- [Download](#scrollTo=DZ578DDKVyQ_&line=2&uniqifier=1) data from Yahoo Finance
- Apply the following preprocessing [steps](#scrollTo=JxhxrrleWbTn&line=2&uniqifier=1):
    1. Convert data types and select year/month
    2. Select the target column
    3. Apply filtering
    4. Aggregate daily data to monthly by selecting the last price of each month
- [Upload](#scrollTo=gxc03uCejWrc&line=2&uniqifier=1) processed data to the repository using the [upload_file function](#scrollTo=AkRCJYSmzsPL&line=14&uniqifier=1)

In [44]:
import requests
import json
from sklearn.pipeline import Pipeline
import pandas as pd
from google.colab import drive
import json
import base64
import yfinance as yf

Declaring constants

In [45]:
#url of a json file from my repo containes all nesessary url from bundesbank's database
API_DATA = 'https://raw.githubusercontent.com/EvgeniyStrizhak/My-master-s-thesis/refs/heads/main/row_datasets/files_info.json'
#json file contains my github token to push files in repo
CONFIG_PATH = "/content/drive/My Drive/config.json"
#repo owner
REPO_OWNER = "EvgeniyStrizhak"
#repo name
REPO_NAME = "My-master-s-thesis"

Select the data period

In [46]:
# Query to filter the dataset for the desired time period
QUERY = 'year >= 2005 and not (year == 2025 and quarter in (2, 3, 4))'

In [47]:
# Mount Google Drive to download GitHub token
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Declaring functions and a data pipeline

In [48]:
# Downloads data from a specified URL and saves it locally
def get_data(url, file_name):
  response = requests.get(url)

# Check if the request was successful
  if response.status_code == 200:
      # Save the file locally
      with open(file_name, "wb") as file:
          file.write(response.content)
      print(f"File downloaded successfully: {file_name}")
      return file_name
  else:
    print(f"Failed to download file: {file_name} Status code: {response.status_code}")
    return None

In [49]:
# Retrieves data from Yahoo Finance and returns it as a DataFrame
def parse_yahoo_data(ticker_text, period_text):
    ticker = yf.Ticker(ticker_text)
    df = ticker.history(period=period_text)

# Convert date column and extract year, month, and quarter
    df['Date'] = pd.to_datetime(df.index)
    df['month'] = df['Date'].dt.month
    df['year'] = df['Date'].dt.year
    df['quarter'] = df['Date'].dt.quarter

# Select the target column and remove irrelevant ones
    df = df.drop(columns=['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'])
    df.rename(columns={'Close': ticker_text}, inplace=True)
    df.reset_index(drop=True, inplace=True)

# Apply the query filter
    df = df.query(QUERY)

# Aggregate daily data to monthly by taking the last price of each month
    df_grouped = df.groupby(['year', 'month']).tail(1).reset_index(drop=True)
    df_grouped = df_grouped.drop(columns=['Date'])
    return df_grouped

Preprocess data from the Bundesbank

In [50]:
#Reads a CSV file into a DataFrame
def read_csv(file_name):
    return pd.read_csv(file_name)

# Removes comment rows and metadata from the top of the file
def filter_rows(df):
    df = df.dropna(subset=['Unnamed: 0'])
    return df.iloc[10:]

# Splits the 'Unnamed: 0' column (which contains date info) into year and quarter/month
def split_date_column(df, period):
    if period == 'quarter':
        return df.assign(
                year=df['Unnamed: 0'].str.split('-', expand=True)[0],
                quarter=df['Unnamed: 0'].str.split('-', expand=True)[1].str.replace('Q', '')
                )
    else:
        df = df.assign(
                year=df['Unnamed: 0'].str.split('-', expand=True)[0],
                month=df['Unnamed: 0'].str.split('-', expand=True)[1],
                )
        # Add quarter column for compatibility with other datasets
        df['quarter'] = pd.to_datetime(df['month'], format='%m').dt.quarter
        return df

# Drops unnecessary columns such as metadata or flags
def drop_unnecessary_columns(df):
    return df.drop(columns=['Unnamed: 0', df.columns[2]])

# Renames the feature column using the file name (excluding .csv)
def rename_columns(df, file_name):
    columns = df.columns
    file_name = file_name.replace('.csv', '')
    return df.rename(columns={columns[0]: file_name})

# Converts year, quarter, and month columns to integers
def convert_dates_to_int(df, period):
    if period == 'month':
        df = df.assign(
                    month=df['month'].astype('int')
                    )
    return df.assign(year=df['year'].astype('int'),
                    quarter=df['quarter'].astype('int')
                    )

# Filters data to include only the defined time range
def filter_by_year(df):
    return df.query(QUERY)

# Converts feature column values to float type
def convert_feature_to_float(df, file_name):
    file_name = file_name.replace('.csv', '')
    df[file_name] = df[file_name].astype('float')
    return df

# Resets the DataFrame index
def reset_index(df):
    df.reset_index(drop=True, inplace=True)
    return df

In [51]:
# Combines all preprocessing functions into a single pipeline
def process_dataset(url, file_name, period):
    df = pd.read_csv(get_data(url, file_name))
    return (df
            .pipe(filter_rows)
            .pipe(split_date_column, period)
            .pipe(drop_unnecessary_columns)
            .pipe(rename_columns, file_name)
            .pipe(convert_dates_to_int, period)
            .pipe(filter_by_year)
            .pipe(convert_feature_to_float, file_name)
            .pipe(reset_index)
            )

Uploads the processed dataset to a GitHub repository

In [52]:
# Uploads the processed dataset to a GitHub repository
def upload_file(df, file_path, github_file_path):
    # Save DataFrame as a local CSV file
    df.to_csv(file_path, index=False)
    # Generate the GitHub API URL
    url = f"https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}/contents/{github_file_path}"
    headers = {"Authorization": f"token {github_token}", "Accept": "application/vnd.github.v3+json"}
    # Check if the file already exists in the repository
    response_sha = requests.get(url, headers=headers)

    # Prepare request payload
    data = {
        "message": "Updated CSV-file",
        "content": "",
        "branch": "main"
    }
    # If file exists, include the SHA for an update
    if response_sha.status_code == 200:
        sha = response_sha.json()["sha"]
        data["sha"] = sha
    else:
        print(f"Error getting SHA: {response_sha.json()}")
        exit(1)

    # Read and encode the file in base64
    with open(file_path, "rb") as file:
        data['content'] = base64.b64encode(file.read()).decode("utf-8")


    # Upload or update the file via GitHub API
    response = requests.put(url, json=data, headers=headers)

    if response.status_code == 201:
        print("A new file has been uploaded succesfully")
    elif response.status_code == 200:
        print("An existing file has been updated succesfully")
    else:
        print(f"Error: {response.status_code} - {response.json()}")

## Feature Uploading and Processing

###Downloading oil prices from Yahoo finance

In [53]:
# Apply the previously defined function to retrieve and preprocess oil price data
oil_price = parse_yahoo_data("CL=F", "21y")

In [54]:
oil_price.head()

,CL=F,month,year,quarter
0,48.200001,1,2005,1
1,51.750000,2,2005,1
2,55.400002,3,2005,1
3,49.720001,4,2005,2
4,51.970001,5,2005,2


In [55]:
oil_price.tail()

,CL=F,month,year,quarter
238,68.000000,11,2024,4
239,71.720001,12,2024,4
240,72.529999,1,2025,1
241,69.760002,2,2025,1
242,71.480003,3,2025,1


In [56]:
oil_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CL=F     243 non-null    float64
 1   month    243 non-null    int32  
 2   year     243 non-null    int32  
 3   quarter  243 non-null    int32  
dtypes: float64(1), int32(3)
memory usage: 4.9 KB


In [57]:
oil_price.describe()

,CL=F,month,year,quarter
count,243.000000,243.000000,243.00000,243.000000
mean,72.117119,6.444444,2014.62963,2.481481
std,21.064901,3.474821,5.85876,1.125708
min,18.840000,1.000000,2005.00000,1.000000
25%,54.764999,3.000000,2010.00000,1.000000
50%,70.639999,6.000000,2015.00000,2.000000
75%,88.430000,9.000000,2020.00000,3.000000
max,140.000000,12.000000,2025.00000,4.000000


The data is clean and correctly filtered.  

There are no null values, incorrect data types, or outliers.

### Download Data from Bundesbank

The GitHub token (stored as a JSON file on my cloud drive) is used to push the preprocessed files to the repository.

In [58]:
# Load GitHub token from a configuration file stored on Google Drive
with open(CONFIG_PATH) as f:
    config = json.load(f)
    github_token = config.get("GITHUB_TOKEN")

if github_token:
    print("GitHub token loaded successfully!")
else:
    print("Error: GitHub token not found!")

GitHub token loaded successfully!


The file `api_data.json` contains all necessary URLs and metadata required to make API requests to the Deutsche Bundesbank.

In [59]:
# Download metadata for API queries from the repository
get_data(API_DATA, 'api_data.json')
with open('api_data.json', 'r') as json_file:
    data_dictionary = json.load(json_file)

File downloaded successfully: api_data.json


Apply the previously defined data pipeline to all URLs listed in `api_data.json`

In [60]:
# Create an empty DataFrame to initialize the merge operation with incoming datasets
df = pd.DataFrame({'year': [], 'quarter': [], 'month':[]})

# Loop through metadata from the JSON file to download and process each dataset
for item in data_dictionary:
    file_path = item['file_name']
    print(file_path)
    # If a dataset contains only quarterly data, replicate the value for each month within the quarter
    if item['period'] == 'quarter':
        keys = ['year', 'quarter']
    else:
        keys = ['year', 'quarter', 'month']
    df = df.merge(process_dataset(item['url'], file_path, item['period']), on = keys, how = 'outer')

production_sector.csv
File downloaded successfully: production_sector.csv
gdp.csv
File downloaded successfully: gdp.csv
consumer_prices.csv
File downloaded successfully: consumer_prices.csv
industrial_production_index.csv
File downloaded successfully: industrial_production_index.csv
interest_rate.csv
File downloaded successfully: interest_rate.csv
economy's_price competitiveness.csv
File downloaded successfully: economy's_price competitiveness.csv
unemployment_rate.csv
File downloaded successfully: unemployment_rate.csv
labour_costs.csv
File downloaded successfully: labour_costs.csv
mutual_funds_sales.csv
File downloaded successfully: mutual_funds_sales.csv
orders-received.csv
File downloaded successfully: orders-received.csv
balance_of_payments.csv
File downloaded successfully: balance_of_payments.csv
shares_sale.csv
File downloaded successfully: shares_sale.csv


Merge the resulting feature set with oil price data

In [61]:
df = df.merge(oil_price, on = ['year', 'month'], how = 'outer')
df = df.drop(columns=['quarter_y'])
df = df.rename(columns={'quarter_x': 'quarter'})

Validate the result

In [62]:
df.head()

,year,quarter,month,production_sector,gdp,consumer_prices,industrial_production_index,interest_rate,economy's_price competitiveness,unemployment_rate,labour_costs,mutual_funds_sales,orders-received,balance_of_payments,shares_sale,CL=F
0,2005,1,1,87.1,83.40,84.5,75.8,2.0,97.0,11.6,70.3,11818.0,73.6,-19072.388,-4426.0,48.200001
1,2005,1,2,85.8,83.40,84.6,75.9,2.0,97.0,11.9,70.3,5485.0,72.3,12614.579,4658.0,51.750000
2,2005,1,3,86.1,83.40,84.8,76.2,2.0,97.0,12.1,70.3,8491.0,73.7,21713.763,1723.0,55.400002
3,2005,2,4,87.4,83.86,85.0,76.5,2.0,95.6,11.9,70.3,4295.0,73.2,17593.788,-3212.0,49.720001
4,2005,2,5,86.5,83.86,85.0,76.4,2.0,95.6,11.9,70.3,3684.0,73.6,6893.840,3144.0,51.970001


In [63]:
df.tail(10)

,year,quarter,month,production_sector,gdp,consumer_prices,industrial_production_index,interest_rate,economy's_price competitiveness,unemployment_rate,labour_costs,mutual_funds_sales,orders-received,balance_of_payments,shares_sale,CL=F
233,2024,2,6,93.4,104.55,129.0,127.6,4.25,94.0,6.0,115.5,10128.0,86.3,10046.447,-1502.0,81.540001
234,2024,3,7,91.5,104.66,129.3,127.8,4.25,93.9,6.0,115.1,11075.0,87.8,43838.372,3370.0,77.910004
235,2024,3,8,92.8,104.66,129.3,128.2,4.25,93.9,6.0,115.1,7802.0,83.0,5238.364,-2514.0,73.550003
236,2024,3,9,91.5,104.66,129.4,127.6,3.65,93.9,6.0,115.1,6286.0,88.8,39012.456,7403.0,68.169998
237,2024,4,10,91.2,104.45,129.9,127.8,3.40,93.5,6.1,116.6,18134.0,88.1,9602.984,6559.0,69.260002
238,2024,4,11,92.2,104.45,129.9,128.7,3.40,93.5,6.1,116.6,16622.0,84.3,32604.540,-2898.0,68.000000
239,2024,4,12,90.9,104.45,130.3,128.6,3.15,93.5,6.1,116.6,27208.0,89.0,46024.781,-3134.0,71.720001
240,2025,1,1,92.1,104.88,130.6,128.2,3.15,93.4,6.2,117.7,25562.0,84.1,14016.318,7644.0,72.529999
241,2025,1,2,91.2,104.88,131.0,127.9,2.90,93.4,6.2,117.7,20919.0,84.1,637.893,6871.0,69.760002
242,2025,1,3,93.3,104.88,131.2,127.0,2.65,93.4,6.3,117.7,12870.0,87.0,69342.817,-2842.0,71.480003


In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 16 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   year                             243 non-null    int64  
 1   quarter                          243 non-null    int64  
 2   month                            243 non-null    int64  
 3   production_sector                243 non-null    float64
 4   gdp                              243 non-null    float64
 5   consumer_prices                  243 non-null    float64
 6   industrial_production_index      243 non-null    float64
 7   interest_rate                    243 non-null    float64
 8   economy's_price competitiveness  243 non-null    float64
 9   unemployment_rate                243 non-null    float64
 10  labour_costs                     243 non-null    float64
 11  mutual_funds_sales               243 non-null    float64
 12  orders-received       

In [65]:
df.describe()

,year,quarter,month,production_sector,gdp,consumer_prices,industrial_production_index,interest_rate,economy's_price competitiveness,unemployment_rate,labour_costs,mutual_funds_sales,orders-received,balance_of_payments,shares_sale,CL=F
count,243.00000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000,243.000000
mean,2014.62963,2.481481,6.444444,97.408230,96.460864,101.739506,94.414815,1.331893,93.518519,6.955556,88.586420,9746.201646,87.165432,14959.557551,3007.473251,72.117119
std,5.85876,1.125708,3.474821,5.988558,6.911831,12.130683,16.058646,1.526306,1.548001,1.764245,13.373575,8657.121712,8.428197,14513.346319,6842.153394,21.064901
min,2005.00000,1.000000,1.000000,78.100000,83.400000,84.500000,75.800000,0.000000,90.100000,4.900000,70.300000,-37827.000000,56.200000,-28449.198000,-25081.000000,18.840000
25%,2010.00000,1.000000,3.000000,94.250000,90.790000,92.050000,85.650000,0.000000,92.600000,5.700000,77.000000,5112.500000,83.750000,6107.434500,-1030.500000,54.764999
50%,2015.00000,2.000000,6.000000,98.800000,96.320000,99.700000,89.400000,0.750000,93.700000,6.500000,86.500000,9530.000000,88.200000,15006.198000,2183.000000,70.639999
75%,2020.00000,3.000000,9.000000,100.900000,103.790000,105.750000,92.200000,2.500000,94.300000,7.700000,99.000000,13236.500000,91.950000,23522.114000,6386.000000,88.430000
max,2025.00000,4.000000,12.000000,108.500000,105.350000,131.200000,142.300000,4.500000,97.300000,12.100000,117.700000,50401.000000,107.500000,69342.817000,28911.000000,140.000000


All features are clean: no null values, no outliers, and all data types are consistent.

Upload the final dataset to GitHub


In [66]:
github_file_path = 'row_datasets/features.csv'
file_path = 'features.csv'
if df.isna().sum().sum() == 0:
  upload_file(df, file_path, github_file_path)
else:
    print("There are missing values in the DataFrame.")

An existing file has been updated succesfully


## Target Processing

In [67]:
# Apply the previously defined function to retrieve and preprocess stock price data
target = parse_yahoo_data('^GDAXI', "21y")

In [68]:
target.head()

,^GDAXI,month,year,quarter
0,4254.850098,1,2005,1
1,4350.490234,2,2005,1
2,4348.770020,3,2005,1
3,4184.839844,4,2005,2
4,4460.629883,5,2005,2


In [69]:
target.tail()

,^GDAXI,month,year,quarter
238,19626.449219,11,2024,4
239,19909.140625,12,2024,4
240,21732.050781,1,2025,1
241,22551.429688,2,2025,1
242,22163.490234,3,2025,1


In [70]:
target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ^GDAXI   243 non-null    float64
 1   month    243 non-null    int32  
 2   year     243 non-null    int32  
 3   quarter  243 non-null    int32  
dtypes: float64(1), int32(3)
memory usage: 4.9 KB


The data is clean and correctly filtered.  

There are no null values, incorrect data types, or outliers.

In [71]:
#Upload the final target dataset to GitHub
github_file_path = 'row_datasets/target.csv'
file_path = 'target.csv'
upload_file(target, file_path, github_file_path)

An existing file has been updated succesfully
